# Requerimientos

In [1]:
# Manejo de la Data
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Graficos
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import re
import string
from wordcloud import WordCloud

# Dashboard
import dash
from dash import dcc, html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

# Carga y procesamiento de la Data

In [2]:
##################################################
################ CARGA DE LA DATA ################
##################################################

# Cargar el archivo CSV en un DataFrame de pandas
data = pd.read_csv("Airlines Reviews and Rating.csv")

##################################################
########## TRANSFORMACION DE LAS FECHAS ##########
##################################################

# Agrega el año 2024 y convierte a formato de fecha
data['Date Flown'] = pd.to_datetime(data['Date Flown'] + '-2023', format='%b-%d-%Y')

# Crea la serie de tiempo
serie_tiempo = pd.Series(data.index, index=data['Date Flown'])
serie_tiempo_por_mes = serie_tiempo.resample('M').count()

##################################################
############## MANEJO DE DUPLICADOS ##############
##################################################

data = data.drop_duplicates()

##################################################
############# DIVISION POR CIUDADES #############
##################################################

# Dividir la columna 'Route' en dos partes: origen y destino
split_routes = data['Route'].str.split(' to ', n=1, expand=True)
data['Origin'] = split_routes[0].str.strip().replace('London Heathrow', 'London')
data['Destination'] = split_routes[1]
split_routes = data['Destination'].str.split(' via ', n=1, expand=True)
data['Destination'] = split_routes[0].str.strip().replace('London Heathrow', 'London')
data['Destination'] = data['Destination'].str.strip().replace('LHR', 'London')
data['Destination'] = data['Destination'].str.strip().replace('Gatwick', 'London')
data['Destination'] = data['Destination'].str.strip().replace('Heathrow', 'London')
data['Destination'] = data['Destination'].str.strip().replace('LGW', 'London')
data['Destination'] = data['Destination'].str.strip().replace('London City', 'London')
data['Destination'] = data['Destination'].str.strip().replace('London Gatwick', 'London')
data['Destination'] = data['Destination'].str.strip().replace('JFK', 'New York')

# Contar la frecuencia de los países
country_counts = data['Destination'].value_counts()

# Obtener el top de países más comunes
df = country_counts.head(20).to_frame()
df = df.reset_index()

# Asignar los países a las ciudades (usando solo una para demostración)
city_to_country = {
    'London': 'United Kingdom',
    'New York': 'United States',
    'Cape Town': 'South Africa',
    'Johannesburg': 'South Africa',
    'Los Angeles': 'United States',
    'Vancouver': 'Canada',
    'Paris': 'France',
    'Dubai': 'United Arab Emirates',
    'Edinburgh': 'United Kingdom',
    'Madrid': 'Spain',
    'Las Vegas': 'United States',
    'Malaga': 'Spain',
    'Hong Kong': 'China',
    'Rome': 'Italy',
    'Singapore': 'Singapore',
    'Amsterdam': 'Netherlands',
    'Bangkok': 'Thailand',
    'Boston': 'United States',
    'Toronto': 'Canada',
    'Miami': 'United States'
}

# Mapear los países a las ciudades
df['Country'] = df['Destination'].map(city_to_country)

# Crear un nuevo DataFrame con la suma de los recuentos por país
df_countries = df.groupby('Country')['count'].sum().reset_index()

##################################################
################ NUBE DE PALABRAS ################
##################################################

# Filtrar las filas donde 'Recommended' es 'yes' y seleccionar solo la columna 'Users Reviews'
reviews_recommended = data.loc[data['Recommended'] == 'yes', 'Users Reviews']
positivo = reviews_recommended.tolist()
# Filtrar las filas donde 'Recommended' es 'no' y seleccionar solo la columna 'Users Reviews'
reviews_no_recommended = data.loc[data['Recommended'] == 'no', 'Users Reviews']
negativo = reviews_no_recommended.tolist()

# Definir la función clean_text con eliminación de palabras específicas
def clean_text(text):
    text = text.lower()  # Transforma el texto a minúsculas
    text = re.sub('\[.*?¿\]\%', ' ', text)  # Remueve texto entre corchetes y porcentaje
    text = re.sub('[%s¿¡]' % re.escape(string.punctuation), ' ', text)  # Remueve signos de puntuación
    text = re.sub('\w*\d\w*', '', text)  # Remueve palabras que contienen dígitos
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('lt', '', text)
    text = re.sub('\r\n', ' ', text)

    # Eliminar palabras específicas
    words_to_remove = ['flight', 'seat', 'ba']  # Agrega aquí las palabras que deseas eliminar
    for word in words_to_remove:
        text = text.replace(word, '')

    # Eliminar la letra "s" que está sola
    text = re.sub(r'\b(s)\b', '', text)
    
    return text

# Convertir las listas de comentarios a texto y aplicar la limpieza
positivo_text = ' '.join([clean_text(comment) for comment in positivo])
negativo_text = ' '.join([clean_text(comment) for comment in negativo])

# Crear una instancia de WordCloud para comentarios positivos
wordcloud_positivo = WordCloud(width=800, height=400, background_color='black').generate(positivo_text)

# Crear una instancia de WordCloud para comentarios negativos
wordcloud_negativo = WordCloud(width=800, height=400, background_color='black').generate(negativo_text)

##################################################
############### ESTUDIO DE CARACTERISTICAS ##############
##################################################

df_features = data.drop(["Seat_Types","Aircraft Type" , "Date Flown" , "Country" , "Type_of_Travellers" , "Route", "Users Reviews", "Origin", "Destination"] , axis = 1)
le = LabelEncoder()
for col in df_features.columns:
    if col == "Recommended":
        pass
    else:
        df_features[col] = le.fit_transform(df_features[col])

##################################################
################## MAPA DE CALOR #################
##################################################

df_2dhist = pd.DataFrame({
    x_label: grp['Seat_Types'].value_counts()
    for x_label, grp in data.groupby('Type_of_Travellers')
})

# Reordenar las filas del DataFrame
nuevo_orden_filas = ['Economy Class', 'Premium Economy', 'Business Class', 'First Class']  # Cambia esto por el orden deseado
df_2dhist = df_2dhist.reindex(nuevo_orden_filas)


<>:94: SyntaxWarning: invalid escape sequence '\['
<>:96: SyntaxWarning: invalid escape sequence '\w'
<>:94: SyntaxWarning: invalid escape sequence '\['
<>:96: SyntaxWarning: invalid escape sequence '\w'
/var/folders/lr/gg3jpj7s6vqft81dzw0pfgz00000gn/T/ipykernel_17449/1905525742.py:94: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?¿\]\%', ' ', text)  # Remueve texto entre corchetes y porcentaje
/var/folders/lr/gg3jpj7s6vqft81dzw0pfgz00000gn/T/ipykernel_17449/1905525742.py:96: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)  # Remueve palabras que contienen dígitos
/var/folders/lr/gg3jpj7s6vqft81dzw0pfgz00000gn/T/ipykernel_17449/1905525742.py:17: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  serie_tiempo_por_mes = serie_tiempo.resample('M').count()


# App

In [3]:
# Definir la aplicación Dash
app = dash.Dash(__name__)

# Estilo de los gráficos
graph_style = {'width': '40%', 'height': '300px'}  # Puedes ajustar el ancho y el alto según tus necesidades
colors = {
    'negro' :'#000000',
    'background': '#111111',
    'text': '#7FDBFF'
}

# Definir los gráficos
# Gráfico de torta
tipo_viajero_counts = data['Type_of_Travellers'].value_counts()
pie_chart = dcc.Graph(
    id='pie-chart',
    figure={
        'data': [go.Pie(
            labels=tipo_viajero_counts.index,
            values=tipo_viajero_counts.values,
            textinfo='label+percent',
            insidetextorientation='radial',
            textposition='inside',
            hole=0.2,
            marker=dict(colors=sns.color_palette("plasma", len(tipo_viajero_counts)))
        )],
        'layout': {
            'title': 'Distribución de tipos de viajeros',
            'plot_bgcolor': '#000000',  # Cambio de color de fondo
            'paper_bgcolor': '#000000',  # Cambio de color de fondo
            'font': {
                'color': colors['text']
            }
        }
    }
)

wordcloud_size = 500
# Nube de palabras para comentarios positivos
wordcloud_positivo = WordCloud(width=wordcloud_size, height=wordcloud_size, background_color='white', colormap='Greens').generate(positivo_text)
positive_wordcloud = html.Div([
    html.H2(children='Nube de palabras para Comentarios Positivos', style={'color': colors['text']}),
    html.Img(src=wordcloud_positivo.to_image(), style={'width': '100%', 'height': 'auto'})
])

# Nube de palabras para comentarios negativos
wordcloud_negativo = WordCloud(width=wordcloud_size, height=wordcloud_size, background_color='white', colormap='Reds').generate(negativo_text)
negative_wordcloud = html.Div([
    html.H2(children='Nube de palabras para Comentarios Negativos', style={'color': colors['text']}),
    html.Img(src=wordcloud_negativo.to_image(), style={'width': '100%', 'height': 'auto'})
])

# Gráfico de serie de tiempo
time_series_graph = dcc.Graph(
    id='time-series-graph',
    figure={
        'data': [go.Scatter(
            x=serie_tiempo_por_mes.index,
            y=serie_tiempo_por_mes.values,
            mode='lines',
            marker=dict(color='#FF851B')
        )],
        'layout': {
            'title': 'Serie de Tiempo de Pasajeros por Mes del 2024',
            'xaxis': {'title': 'Mes'},
            'yaxis': {'title': 'Número de Pasajeros'},
            'plot_bgcolor': '#000000',  # Cambio de color de fondo
            'paper_bgcolor': '#000000',  # Cambio de color de fondo
            'font': {
                'color': colors['text']
            }
        }
    }
)


# Mapa de calor
heatmap = dcc.Graph(
    id='heatmap',
    figure={
        'data': [go.Heatmap(
            z=df_2dhist.values,
            y=df_2dhist.index,
            x=df_2dhist.columns,
            colorscale='Plasma'
        )],
        'layout': {
            'title': 'Relación tipo de asiento vs tipo de pasajero',
            'yaxis': {'title': 'Seat Types', 'tickangle': -20},
            'xaxis': {'title': 'Type of Travellers', 'tickangle': -30},
            'plot_bgcolor': '#000000',  # Cambio de color de fondo
            'paper_bgcolor': '#000000',  # Cambio de color de fondo
            'font': {
                'color': colors['text']
            },
            'margin': {'l': 150,'t': 50, 'b': 100}  # Ajuste de los márgenes superior e inferior
        }
    }
)

# Barplot top 10 destinos
df_sorted = df_countries.sort_values(by='count', ascending=True)
df_barplot = df_sorted.drop(df_sorted[df_sorted['Country'].isin(['United Kingdom', 'Italy'])].index)
df_barplot.set_index('Country', inplace=True)

barplot_graph = dcc.Graph(
    id='barplot-graph',
    figure={
        'data': [go.Bar(
            x=df_barplot.index,
            y=df_barplot['count'],
            marker=dict(color='#FF851B'),
        )],
        'layout':{
            'title':'Top 10 destinos preferidos excluyendo al Reino Unido',
            'xaxis':{'title':'Country'},
            'yaxis':{'title':'Values'},
            'plot_bgcolor': '#000000',  # Cambio de color de fondo
            'paper_bgcolor': '#000000',  # Cambio de color de fondo
            'font': {
                'color': colors['text']
            },
            'margin': {'l': 150,'t': 50, 'b': 100}  # Ajuste de los márgenes superior e inferior
        }
    }
)

# Recomendacion vs Caracteristicas
# Define una lista de opciones para el dropdown
###################################################### CAMBIAR COLOR DE FONDO #############################################
dropdown_options = [{'label': col, 'value': col} for col in df_features.columns[:-1]] # Excluye la columna 'Recommended'



# Crear el layout del dashboard
# Agregar los gráficos al layout con la disposición deseada utilizando Flexbox
app.layout = html.Div(style={'backgroundColor': colors['negro'], 'padding': '20px'}, children=[
    html.H1(
        children='Airlines: Análisis de Datos de Viajes',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'marginBottom': '20px'  # Añade margen inferior al título
        }
    ),
    html.Div(
        children='El conjunto de datos Airlines proporciona una amplia colección de comentarios de pasajeros sobre su experiencia de vuelo,  ofreciendo información sobre la satisfacción del cliente y la calidad del servicio.',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'marginBottom': '20px'  # Añade margen inferior al texto
        }
    ),
    html.Div(
        children='Objetivo: Desplegar un Dashboard que permita identificar las variables principales que influyen en una buena/mala experiencia del usuario durante un vuelo comercial.',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'marginBottom': '40px'  # Añade margen inferior al texto
        }
    ),

    # Primera Fila
    html.Div([
        html.Div([
            dcc.Dropdown(
            id='dropdown',
            options=dropdown_options,
            value=dropdown_options[0]['value'],  # Valor predeterminado del dropdown
            style={'flex': '100%', 'width':'90vw'}
            )
        ])
    ], style={'display': 'flex', 'justifyContent': 'center', 'width':'100%'}),

    html.Div([
        html.Div([
            dcc.Graph(
            id='output-graph',
            style={'flex': '100%', 'width':'90vw'}
            )
        ])
    ], style={'display': 'flex', 'justifyContent': 'center', 'width':'100%'}),

    html.Div([
        html.Div(
            children='El gráfico de barras muestra la distribución de las calificaciones dadas por los usuarios a los distintos servicios encuestados, agrupados en "yes" y "no" si el encuestado recomendaba la aerolinea o no.',
            style={
                'textAlign': 'center',
                'font-size':'1.2rem',
                'color': 'black',
                'background-color':'white',
                'margin':'0px',
                'marginBottom': '20px',  # Añade margen inferior al texto
                'width':'90vw'
            }
        )
    ], style={'display': 'flex', 'justifyContent': 'center', 'width':'100%'}),

    # Segunda fila
    html.Div([
        html.Div([negative_wordcloud], style={'flex': '50%', 'justifyContent': 'center', 'margin':'10px'}),
        html.Div([positive_wordcloud], style={'flex': '50%', 'justifyContent': 'center', 'margin':'10px'})
    ], style={'display': 'flex', 'justifyContent': 'center', 'marginLeft':'50px', 'marginBottom': '20px', 'width': '90vw'}),

    # Tercera fila
    html.Div([
        html.Div([pie_chart], style={'flex': '40%', 'marginRight': '10px'}),
        html.Div([time_series_graph], style={'flex': '60%'})
    ], style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '20px'}),

    # Cuarta fila
    html.Div([
        html.Div([barplot_graph], style={'flex': '60%'}),
        html.Div([heatmap], style={'flex': '40%'}),
    ], style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '20px'})
])



@app.callback(
    Output('output-graph', 'figure'),
    [Input('dropdown', 'value')]
)

# Funcion para crear gráficos de características
def create_bar_chart(selected_column):
    # Agrupar los datos por 'Recommended' y 'selected_column'
    count_data = df_features.groupby(['Recommended', selected_column]).size().unstack()
    
    # Crear un gráfico de barras
    fig = go.Figure()
    


    ########### Modulo paleta de colores ###########

    # Elegir una paleta de colores progresiva
    color_palette_no = px.colors.sequential.Reds_r
    color_palette_yes = px.colors.sequential.Greens
    


    ########### Modulo diferenciación por etiquetas ###########
    
    # Agregar barras para la etiqueta "no"
    if 'no' in count_data.index:
        for i, column in enumerate(count_data.columns):
            fig.add_trace(go.Bar(
                x=['no'],  # Solo una etiqueta para "no"
                y=[count_data.loc['no', column]],  # Altura de la barra para "no"
                name=column,  # Nombre de la barra (será mostrado en la leyenda)
                marker=dict(color=color_palette_no[i]),  # Asignar color de la paleta para "no"
            ))
    
    # Agregar barras para la etiqueta "si"
    if 'yes' in count_data.index:
        for i, column in enumerate(count_data.columns):
            fig.add_trace(go.Bar(
                x=['yes'],  # Solo una etiqueta para "si"
                y=[count_data.loc['yes', column]],  # Altura de la barra para "si"
                name=column,  # Nombre de la barra (será mostrado en la leyenda)
                marker=dict(color=color_palette_yes[i]),  # Asignar color de la paleta para "si"
            ))




    ########### Modulo Layout
    # Modificar el layout del gráfico
    fig.update_layout(
        title=f'{selected_column} Rating',
        xaxis_title='Recommended',
        yaxis_title=selected_column,
        barmode='group',  # Barras agrupadas en lugar de apiladas
        legend_title='Rating',  # Título de la leyenda
        legend=dict(
            orientation='v',  # Orientación vertical
            x=1.05,  # Posición en el eje x
            y=0.5,  # Posición en el eje y
            bgcolor='rgba(255, 255, 255, 0.5)',  # Color de fondo de la leyenda con transparencia
            bordercolor='#000000',  # Color del borde de la leyenda
            borderwidth=1  # Ancho del borde de la leyenda
        ),
        annotations=[
            dict(
                x=1.065,  # Posición en el eje x
                y=1.05,  # Posición en el eje y
                xref='paper',
                yref='paper',
                text='No',  # Etiqueta para "no"
                showarrow=False,
                xanchor='left',
                yanchor='middle',
                font=dict(
                    color='#FF0000'  # Color rojo para "no"
                )
            ),
            dict(
                x=1.065,  # Posición en el eje x
                y=-0.05,  # Posición en el eje y
                xref='paper',
                yref='paper',
                text='Yes',  # Etiqueta para "yes"
                showarrow=False,
                xanchor='left',
                yanchor='middle',
                font=dict(
                    color='#008000'  # Color verde para "yes"
                )
            )
        ],
        
    )

    
    return fig

if __name__ == '__main__':
    #app.run_server(debug=True)
    app.run_server(debug=True, use_reloader=False)


In [4]:
app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
